In [1]:
# Read the CSV file
input_path = '/home/iceberg/warehouse'

df_read = spark.read.format('csv').options(header='true').load(f'{input_path}/appsflyer/id123456789/install.csv')

# Show the data
df_read.show()

26/01/07 09:48:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------------+---------------------+-------------------+-------------------+----------+------------+-----------------+-----------+--------+--------------------+------------------+--------------------+------------------+--------------------+------------------+---------------+--------------------+-------------+-------------+----------------+-------------------+-------------------------+---------------------+-----------------------+-----------------------+-------------------+-------------------------+---------------------+-----------------------+-----------------------+-------------------+-------------------------+---------------------+-----------------------+-----------------------+------+------------+-----+--------------------+-----------+---------------+-----+--------------------+------------+----------+--------------------+--------------+----+----------+----------------+----+----+--------+-----------+----------+-----------+-----------+--------------------+--------------------